In [14]:
#Import Required Libraries
import streamlit as st
from azure.core.credentials import AzureKeyCredential
from langchain.text_splitter import RecursiveCharacterTextSplitter
import numpy as np
import faiss
from langchain_community.vectorstores import FAISS
from langchain_core.vectorstores import VectorStoreRetriever
from langchain.chains import RetrievalQA
from langchain.llms import AzureOpenAI
from langchain_openai import AzureOpenAIEmbeddings
from langchain.prompts import PromptTemplate
from langchain.chains.question_answering import load_qa_chain
from langchain_core.messages import HumanMessage
from langchain_openai import AzureChatOpenAI
import openai
import pyodbc
import urllib
from sqlalchemy import create_engine
import pandas as pd
from azure.identity import InteractiveBrowserCredential
from pandasai import SmartDataframe
import pandas as pd
# from pandasai.llm import AzureOpenAI
import matplotlib.pyplot as plt
import os
import time
from PIL import Image
import base64
import pandasql as ps
os.environ['KMP_DUPLICATE_LIB_OK'] = 'TRUE'

#Initializing API Keys to use LLM
AZURE_OPENAI_API_KEY = '17299c5927e64bb382352a6e246fa6ef'
AZURE_OPENAI_ENDPOINT = 'https://fordmustang.openai.azure.com/'


#Reading the dataset
Sentiment_Data  = pd.read_csv("Sampled_Copilot_Reviews_Final.csv")

#Function to derive Sentiment Score based on Sentiment
def Sentiment_Score_Derivation(value):
    try:
        if value == "positive":
            return 1
        elif value == "negative":
            return -1
        else:
            return 0
    except Exception as e:
        err = f"An error occurred while deriving Sentiment Score: {e}"
        return err    

#Deriving Sentiment Score and Review Count columns into the dataset
Sentiment_Data["Sentiment_Score"] = Sentiment_Data["Sentiment"].apply(Sentiment_Score_Derivation)
Sentiment_Data["Review_Count"] = 1.0


################################# Definiting Functions #################################

#Review Summarization (Detailed) + Feature Comparison and Suggestion

#Function to extract text from file
def get_text_from_file(txt_file):
    try:
        with open(txt_file, 'r',encoding='latin') as file:
            text = file.read()
        return text
    except Exception as e:
        err = f"An error occurred while getting text from file: {e}"
        return err

# Function to split text into chunks
def get_text_chunks(text):
    try:
        text_splitter = RecursiveCharacterTextSplitter(chunk_size=10000, chunk_overlap=1000)
        chunks = text_splitter.split_text(text)
        return chunks
    except Exception as e:
        err = f"An error occurred while getting text chunks: {e}"
        return err

# Function to create and store embeddings
def get_vector_store(text_chunks):
    try:
        embeddings = AzureOpenAIEmbeddings(azure_deployment="MV_Agusta")
        vector_store = FAISS.from_texts(text_chunks, embedding=embeddings)
        vector_store.save_local("faiss_index_CopilotSample")
        return vector_store
    except Exception as e:
        err = f"An error occurred while getting vectos: {e}"
        return err

# Function to setup the vector store (to be run once or upon text update)
def setup(txt_file_path):
    try:
        raw_text = get_text_from_file(txt_file_path)
        text_chunks = get_text_chunks(raw_text)
        get_vector_store(text_chunks)
        print("Setup completed. Vector store is ready for queries.")
    except Exception as e:
        err = f"An error occurred while setting up vector store: {e}"
        return err

# Function to get conversational chain
def get_conversational_chain_detailed(history):
    try:
        hist = """"""
        for i in history:
            hist = hist+"\nUser: "+i[0]
            if isinstance(i[1],pd.DataFrame):
                x = i[1].to_string()
            else:
                x = i[1]
            hist = hist+"\nResponse: "+x
        prompt_template = """
        Given a dataset with these columns: Review, Data_Source, Geography, Product_Family, Sentiment and Aspect (also called Features)
          
          Review: This column contains the opinions and experiences of users regarding different product families across geographies, providing insights into customer satisfaction or complaints and areas for improvement.
          Data_Source: This column indicates the platform from which the user reviews were collected, such as Reddit, Play Store, App Store, Tech Websites, or YouTube videos.
          Geography: This column lists the countries of the users who provided the reviews, allowing for an analysis of regional preferences and perceptions of the products.
          Product_Family: This column identifies the broader category of products to which the revie
w pertains, enabling comparisons and trend analysis across different product families.
          Sentiment: This column reflects the overall tone of the review, whether positive, negative, or neutral, and is crucial for gauging customer sentiment.
          Aspect: This column highlights the particular features or attributes of the product that the review discusses, pinpointing areas of strength or concern.
          
          Perform the required task from the list below, as per user's query: 
          1. Review Summarization - Summarize the reviews by filtering the relevant Aspect, Geography, Product_Family, Sentiment or Data_Source, only based on available reviews and their sentiments in the dataset.
          2. Aspect Comparison - Provide a summarized comparison for each overlapping feature/aspect between the product families or geographies ,  only based on available user reviews and their sentiments in the dataset. Include pointers for each aspect highlighting the key differences between the product families or geographies, along with the positive and negative sentiments as per customer perception.
          3. New Feature Suggestion/Recommendation - Generate feature suggestions or improvements or recommendations based on the frequency and sentiment of reviews and mentioned aspects and keywords. Show detailed responses to user queries by analyzing review sentiment, specific aspects, and keywords.
          4. Hypothetical Reviews - Based on varying customer sentiments for the reviews in the existing dataset, generate hypothetical reviews for any existing feature updation or new feature addition in any device family across any geography, by simulating user reactions. Ensure to synthesize realistic reviews that capture all types of sentiments and opinions of users, by considering their hypothetical prior experience working with the new feature and generate output based on data present in dataset only. After these, provide solutions/remedies for negative hypothetical reviews. 
          
          Enhance the model’s comprehension to accurately interpret user queries by:
          Recognizing abbreviations for country names (e.g., ‘DE’ for Germany, ‘USA’or 'usa' or 'US' for the United States of America) and expanding them to their full names for clarity.
          Understanding product family names even when written in reverse order or missing connecting words (e.g., ‘copilot in windows 11’ as ‘copilot windows’ and ‘copilot for security’ as ‘copilot security’ etc.).
          Utilizing context and available data columns to infer the correct meaning and respond appropriately to user queries involving variations in product family names or geographical references
          Please provide a comprehensive Review summary, feature comparison, feature suggestions for specific product families and actionable insights that can help in product development and marketing strategies.
          Generate acurate response only, do not provide extra information.
            
            Important: Generate outputs using the provided dataset only, don't use pre-trained information to generate outputs.\n Following is the previous conversation from User and Response, use it to get context only:""" + hist + """\n
                Use the above conversation chain to gain context if the current prompt requires context from previous conversation.\n
        Context:\n {context}?\n
        Question: \n{question}\n

        Answer:
        """
        prompt = PromptTemplate(template=prompt_template, input_variables=["context", "question"])
        model = AzureChatOpenAI(
            azure_deployment="Thruxton_R",
            api_version='2024-03-01-preview',
            temperature = 0.4)
        chain = load_qa_chain(model, chain_type="stuff", prompt=prompt)
        return chain
    except Exception as e:
        err = f"An error occurred while getting conversation chain for detailed review summarization: {e}"
        return err

# Function to handle user queries using the existing vector store
def query_detailed(user_question, history, vector_store_path="faiss_index_CopilotSample"):
    try:
        embeddings = AzureOpenAIEmbeddings(azure_deployment="MV_Agusta")
        vector_store = FAISS.load_local(vector_store_path, embeddings, allow_dangerous_deserialization=True)
        chain = get_conversational_chain_detailed(history)
        docs = vector_store.similarity_search(user_question)
        response = chain({"input_documents": docs, "question": user_question}, return_only_outputs=True)
        return response["output_text"]
    except Exception as e:
        err = f"An error occurred while getting LLM response for detailed review summarization: {e}"
        return err


## Review Summarization (Quantifiable)

#Converting Top Operator to Limit Operator as pandasql doesn't support Top
def convert_top_to_limit(sql):
    try:
        tokens = sql.upper().split()
        is_top_used = False

        for i, token in enumerate(tokens):
            if token == 'TOP':
                is_top_used = True
                if i + 1 < len(tokens) and tokens[i + 1].isdigit():
                    limit_value = tokens[i + 1]
                    # Remove TOP and insert LIMIT and value at the end
                    del tokens[i:i + 2]
                    tokens.insert(len(tokens), 'LIMIT')
                    tokens.insert(len(tokens), limit_value)
                    break  # Exit loop after successful conversion
                else:
                    raise ValueError("TOP operator should be followed by a number")

        return ' '.join(tokens) if is_top_used else sql
    except Exception as e:
        err = f"An error occurred while converting Top to Limit in SQL Query: {e}"
        return err

#Function to add Table Name into the SQL Query as it is, as the Table Name is Case Sensitive here
def process_tablename(sql, table_name):
    try:
        x = sql.upper()
        query = x.replace(table_name.upper(), table_name)
        return query
    except Exception as e:
        err = f"An error occurred while processing table name in SQL query: {e}"
        return err

## Generating Response by Identifying Prompt Nature

#Function to get conversation chain for quantitative outputs and also add context from historical conversation as well
def get_conversational_chain_quant(history):
    try:
        hist = """"""
        for i in history:
            hist = hist+"\nUser: "+i[0]
            if isinstance(i[1],pd.DataFrame):
                x = i[1].to_string()
            else:
                x = i[1]
            hist = hist+"\nResponse: "+x
        prompt_template = """
        1. Your Job is to convert the user question to SQL Query (Follow Microsoft SQL server SSMS syntax.). You have to give the query so that it can be used on Microsoft SQL server SSMS.You have to only return query as a result.
            2. There is only one table with table name Sentiment_Data where each row is a user review. The table has 10 columns, they are:
                Review: Review of the Copilot Product
                Data_Source: From where is the review taken. It contains following values: 'LaptopMag', 'PCMag', 'Verge', 'ZDNET', 'PlayStore', 'App Store','AppStore', 'Reddit', 'YouTube'.
                Geography: From which Country or Region the review was given. It contains following values: 'Unknown', 'Brazil', 'Australia', 'Canada', 'China', 'Germany','France'.
                Title: What is the title of the review
                Review_Date: The date on which the review was posted
                Product: Corresponding product for the review. It contains following values: 'COPILOT'.
                Product_Family: Which version or type of the corresponding Product was the review posted for. It contains following values: 'Copilot in Windows 11', 'Copilot for Microsoft 365','Microsoft Copilot', 'Copilot for Security', 'Copilot Pro','Github Copilot', 'Copilot for Mobile'.
                Sentiment: What is the sentiment of the review. It contains following values: 'positive', 'neutral', 'negative'.
                Aspect: The review is talking about which aspect or feature of the product. It contains following values: 'Microsoft Product', 'Interface', 'Connectivity', 'Privacy','Compatibility', 'Generic', 'Innovation', 'Reliability','Productivity', 'Price', 'Text Summarization/Generation','Code Generation', 'Ease of Use', 'Performance','Personalization/Customization'.
                Keyword: What are the keywords mentioned in the product
                Review_Count - It will be 1 for each review or each row
                Sentiment_Score - It will be 1, 0 or -1 based on the Sentiment.
            3. Sentiment mark is calculated by sum of Sentiment_Score.
            4. Net sentiment is calculcated by sum of Sentiment_Score divided by sum of Review_Count. It should be in percentage. Example:
                    SELECT ((SUM(Sentiment_Score)*1.0)/(SUM(Review_Count)*1.0)) * 100 AS Net_Sentiment 
                    FROM Sentiment_Data
                    ORDER BY Net_Sentiment DESC
            5. Net sentiment across country or across region is sentiment mark of a country divided by total reviews of that country. It should be in percentage.
                Example to calculate net sentiment across country:
                    SELECT Geography, ((SUM(Sentiment_Score)*1.0) / (SUM(Review_Count)*1.0)) * 100 AS Net_Sentiment
                    FROM Sentiment_Data
                    GROUP BY Geography
                    ORDER BY Net_Sentiment DESC
            6. Net Sentiment across a column "X" is calculcated by Sentiment Mark for each "X" divided by Total Reviews for each "X".
                Example to calculate net sentiment across a column "X":
                    SELECT X, ((SUM(Sentiment_Score)*1.0) / (SUM(Review_Count)*1.0)) * 100 AS Net_Sentiment
                    FROM Sentiment_Data
                    GROUP BY X
                    ORDER BY Net_Sentiment DESC
            7. Distribution of sentiment is calculated by sum of Review_Count for each Sentiment divided by overall sum of Review_Count
                Example: 
                    SELECT Sentiment, SUM(ReviewCount)*100/(SELECT SUM(Review_Count) AS Reviews FROM Sentiment_Data) AS Total_Reviews 
                    FROM Sentiment_Data 
                    GROUP BY Sentiment
                    ORDER BY Total_Reviews DESC
            8. Convert numerical outputs to float upto 1 decimal point.
            9. Always include ORDER BY clause to sort the table based on the aggregate value calculated in the query.
            10. Top Country is based on Sentiment_Score i.e., the Country which have highest sum(Sentiment_Score)
            11. Always use 'LIKE' operator whenever they mention about any Country. Use 'LIMIT' operator instead of TOP operator.Do not use TOP OPERATOR. Follow syntax that can be used with pandasql.
            12. If you are using any field in the aggregate function in select statement, make sure you add them in GROUP BY Clause.
            13. Make sure to Give the result as the query so that it can be used on Microsoft SQL server SSMS.
            14. Important: Always show Net_Sentiment in Percentage upto 1 decimal point. Hence always make use of ROUND function while giving out Net Sentiment and Add % Symbol after it.
            15. Important: User can ask question about any categories including Aspects, Geograpgy, Sentiment etc etc. Hence, include the in SQL Query if someone ask it.
            16. Important: You Response should directly starts from SQL query nothing else.
            17. Important: Always use LIKE keyword instead of = symbol while generating SQL query.
            18. Important: Generate outputs using the provided dataset only, don't use pre-trained information to generate outputs.
        \n Following is the previous conversation from User and Response, use it to get context only:""" + hist + """\n
                Use the above conversation chain to gain context if the current prompt requires context from previous conversation.\n
        Context:\n {context}?\n
        Question: \n{question}\n

        Answer:
        """
        prompt = PromptTemplate(template=prompt_template, input_variables=["context", "question"])
        model = AzureChatOpenAI(
            azure_deployment="Thruxton_R",
            api_version='2024-03-01-preview',
            temperature = 0.3)
        chain = load_qa_chain(model, chain_type="stuff", prompt=prompt)
        return chain
    except Exception as e:
        err = f"An error occurred while getting conversation chain for quantifiable review summarization: {e}"
        return err

#Function to convert user prompt to quantitative outputs for Copilot Review Summarization
def query_quant(user_question, history, vector_store_path="faiss_index_CopilotSample"):
    try:
        # Initialize the embeddings model
        embeddings = AzureOpenAIEmbeddings(azure_deployment="MV_Agusta")
        
        # Load the vector store with the embeddings model
        vector_store = FAISS.load_local(vector_store_path, embeddings, allow_dangerous_deserialization=True)
        
        # Rest of the function remains unchanged
        chain = get_conversational_chain_quant(history)
        docs = []
        response = chain({"input_documents": docs, "question": user_question}, return_only_outputs=True)
        SQL_Query = response["output_text"]
        SQL_Query = convert_top_to_limit(SQL_Query)
        SQL_Query = process_tablename(SQL_Query,"Sentiment_Data")
    #     print(SQL_Query)
        data = ps.sqldf(SQL_Query, globals())
        data_1 = data
        html_table = data.to_html(index=False)
    #     return html_table
        return data_1
    except Exception as e:
        err = f"An error occurred while generating response for quantitative review summarization: {e}"
        return err


## Generating Response by Identifying Prompt Nature


#Function to identify the nature of prompt, whether the user is asking for a detailed summary or a quantitative summary
def identify_prompt(user_question):
    try:
        prompt_template = """
        Given a user prompt about customer reviews for products (Copilot, Windows, Surface) and various different features, classify the prompt into one of two categories:
            Quantifiable: This prompt seeks a numerical answer or data point related to the reviews. 
                            (e.g., "What is the net sentiment score for Product A reviews?", 
                                    "How many reviews mention the battery life of Product B?", 
                                    "Calculate the net sentiment of Product A.", 
                                    "Net Sentiment", 
                                    "Sentiment Score", 
                                    "Top Countries", 
                                    "Top Products", etc.)
            Detailed: This prompt seeks a summary, comparison, recommendation/suggestion or hypothetical reviews based on the reviews, expressed in words. The task can be either Review Summarization, Feature Comparison, New Feature Suggestion/Recommendation or Hypothetical Reviews generation based on the type of user question:
                      (example of Review Summarization - Summarize / Give a summary of the reviews for different product families or geographies, 
                      example of Aspect Comparison - Give feature comparison among product families a1,a2,a3... across geographies g1,g2,g3... or Compare the features of product families a1,a2,a3... across geographies g1,g2,g3,... or Compare utility of feature 'x' among product families a1,a2,a3... across geographies g1,g2,g3... , 
                      example of New Feature Suggestion/Recommendation - Suggest new features or improvements or recommendations for different product families in different geographies,
                      example of Hypothetical Reviews - Generate hypothetical user reviews for the feature upgrade for any product family in any geography, focusing on the feature/aspect or Provide hypothetical user reviews for the addition of the new feature 'x' in any product family across any geography)

        Input: User prompt about customer reviews
        Output: Category (Quantifiable or Detailed)
        Context:\n {context}?\n
        Question: \n{question}\n

        Answer:
        """
        prompt = PromptTemplate(template=prompt_template, input_variables=["context", "question"])
        model = AzureChatOpenAI(
            azure_deployment="Thruxton_R",
            api_version='2024-03-01-preview')
        chain = load_qa_chain(model, chain_type="stuff", prompt=prompt)
        response = chain({"input_documents": [], "question": user_question}, return_only_outputs=True)
        if "detailed" in response["output_text"].lower():
            return "Detailed"
        elif "quantifiable" in response["output_text"].lower():
            return "Quantifiable"
        else:
            return "Others"+"\nPrompt Identified as:"+response["output_text"]+"\n"
    except Exception as e:
        err = f"An error occurred while generating conversation chain for identifying nature of prompt: {e}"
        return err

#Function to generate Review Summarization (Detailed)/Feature Comparison/Feature Suggestion from User Prompt
def review_summarization(user_question, history):
    try:
        txt_file_path = "Sample_Copilot_Reviews_50K_Translated_v2.txt"
        # Automatically call setup with the predefined file on startup
        if not os.path.exists("faiss_index_CopilotSample"):
            setup(txt_file_path)

        if os.path.exists("faiss_index_CopilotSample"):
            response = query_detailed(user_question, history)
            return response
        else:
            return "The vector store setup has failed. Please check the file path and try again."
    except Exception as e:
        err = f"An error occurred while generating detailed review summarization: {e}"
        return err

#Function to generate Quantitative Review Summarization from User Prompt
def quantifiable_data(user_question, history):
    try:
        response = query_quant(user_question, history)
        
        return response
    except Exception as e:
        err = f"An error occurred while generating quantitative review summarization: {e}"
        return err

#Function to generate a response from User Question
def device_llm_review_generator(user_question, history):
    try:
        identity_prompt = identify_prompt(user_question)
        if identity_prompt == "Detailed":
            output = review_summarization(user_question, history)
        elif identity_prompt == "Quantifiable":
            output = quantifiable_data(user_question, history)

            if output.isnull().all().all() or output.empty:
                output = "I don't have necessary knowledge to provide a helpful response. Please ask anything related to Insights around Copilot customer feedback and I'll do my best to assist you."
                
        else:
            output = "Error: Cannot identify the nature of your question\nPrompt identified as: "+identity_prompt
        return output
    except Exception as e:
        err = f"An error occurred while generating LLM response: {e}"
        return err


################################# Model Deployment #################################

def main():
    try:
    # Chat history state management
        if 'chat_history' not in st.session_state:
            st.session_state['chat_history'] = []

        # Create a container for logos and title with horizontal layout
        col1, col2, col3 = st.columns([1, 2, 1])
      
        # Display logo on the left
        with col1:
            st.image("microsoft_logo.png", width=50)  # Adjust width as needed

        # Display title in the center
        with col2:
            st.header("Copilot LLM Review Generator")

        # Display logo on the right
        with col3:
            st.image("copilot_logo.svg", width=50)  # Align the logo to the right
      
        # User input section
        user_input = st.text_input("Enter your text:", placeholder="What would you like to process?")

        # Process button and output section
        if st.button("Process"):
            # Re-check if 'chat_history' is initialized before appending
            if 'chat_history' not in st.session_state:
                st.session_state['chat_history'] = []
            
            output = device_llm_review_generator(user_input,st.session_state['chat_history'])
            st.session_state['chat_history'].append((user_input, output))
        
            # Display output based on type (string or dataframe)
            if isinstance(output, pd.DataFrame):
                st.dataframe(output)
                
            else:
                st.write(output)

        # Chat history section with some formatting
        st.header("Chat History")
        for user_text, output_text in st.session_state['chat_history']:
            st.markdown(f"- You: {user_text}")
            if isinstance(output_text, pd.DataFrame):
                st.dataframe(output_text)  # Convert dataframe to string for display
            else:
                st.markdown(f"- Bot: {output_text}")
            st.write("---")
    except Exception as e:
        err = f"An error occurred while calling the final function: {e}"
        print(err)
        return err

In [3]:
def main():
    try:
    # Chat history state management
        if 'chat_history' not in st.session_state:
            st.session_state['chat_history'] = []

        # Create a container for logos and title with horizontal layout
        col1, col2, col3 = st.columns([1, 2, 1])
      
        # Display logo on the left
        with col1:
            st.image("microsoft_logo.png", width=50)  # Adjust width as needed

        # Display title in the center
        with col2:
            st.header("Copilot LLM Review Generator")

        # Display logo on the right
        with col3:
            st.image("copilot_logo.svg", width=50)  # Align the logo to the right
      
        # User input section
        user_input = st.text_input("Enter your text:", placeholder="What would you like to process?")
        if st.button("Process"):
            st.image("https://in-files.apjonlinecdn.com/landingpages/content-pages/visid-rich-content/hp-laptop-15/images/w100_product_highlight_v1.png",width = 100)
            st.write("HP Laptop 15")
    except Exception as e:
        err = f"An error occurred while calling the final function: {e}"
        print(err)
        return err

In [4]:
main()

2024-05-15 19:32:08.367 WARNING streamlit.runtime.state.session_state_proxy: Session state does not function when running a script without `streamlit run`
2024-05-15 19:32:10.300 
  command:

    streamlit run C:\Anaconda 3\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]


In [5]:
prompt_template = """
        1. Your Job is to convert the user question to SQL Query (Follow Microsoft SQL server SSMS syntax.). You have to give the query so that it can be used on Microsoft SQL server SSMS.You have to only return query as a result.
            2. There is only one table with table name Sentiment_Data where each row is a user review. The table has 10 columns, they are:
                Review: Review of the Copilot Product
                Data_Source: From where is the review taken. It contains following values: 'LaptopMag', 'PCMag', 'Verge', 'ZDNET', 'PlayStore', 'App Store','AppStore', 'Reddit', 'YouTube'.
                Geography: From which Country or Region the review was given. It contains following values: 'Unknown', 'Brazil', 'Australia', 'Canada', 'China', 'Germany','France'.
                Title: What is the title of the review
                Review_Date: The date on which the review was posted
                Product: Corresponding product for the review. It contains following values: 'COPILOT'.
                Product_Family: Which version or type of the corresponding Product was the review posted for. It contains following values: 'Copilot in Windows 11', 'Copilot for Microsoft 365','Microsoft Copilot', 'Copilot for Security', 'Copilot Pro','Github Copilot', 'Copilot for Mobile'.
                Sentiment: What is the sentiment of the review. It contains following values: 'positive', 'neutral', 'negative'.
                Aspect: The review is talking about which aspect or feature of the product. It contains following values: 'Microsoft Product', 'Interface', 'Connectivity', 'Privacy','Compatibility', 'Generic', 'Innovation', 'Reliability','Productivity', 'Price', 'Text Summarization/Generation','Code Generation', 'Ease of Use', 'Performance','Personalization/Customization'.
                Keyword: What are the keywords mentioned in the product
                Review_Count - It will be 1 for each review or each row
                Sentiment_Score - It will be 1, 0 or -1 based on the Sentiment.
            3. Sentiment mark is calculated by sum of Sentiment_Score.
            4. Net sentiment is calculcated by sum of Sentiment_Score divided by sum of Review_Count. It should be in percentage. Example:
                    SELECT ((SUM(Sentiment_Score)*1.0)/(SUM(Review_Count)*1.0)) * 100 AS Net_Sentiment 
                    FROM Sentiment_Data
                    ORDER BY Net_Sentiment DESC
            5. Net sentiment across country or across region is sentiment mark of a country divided by total reviews of that country. It should be in percentage.
                Example to calculate net sentiment across country:
                    SELECT Geography, ((SUM(Sentiment_Score)*1.0) / (SUM(Review_Count)*1.0)) * 100 AS Net_Sentiment
                    FROM Sentiment_Data
                    GROUP BY Geography
                    ORDER BY Net_Sentiment DESC
            6. Net Sentiment across a column "X" is calculcated by Sentiment Mark for each "X" divided by Total Reviews for each "X".
                Example to calculate net sentiment across a column "X":
                    SELECT X, ((SUM(Sentiment_Score)*1.0) / (SUM(Review_Count)*1.0)) * 100 AS Net_Sentiment
                    FROM Sentiment_Data
                    GROUP BY X
                    ORDER BY Net_Sentiment DESC
            7. Distribution of sentiment is calculated by sum of Review_Count for each Sentiment divided by overall sum of Review_Count
                Example: 
                    SELECT Sentiment, SUM(ReviewCount)*100/(SELECT SUM(Review_Count) AS Reviews FROM Sentiment_Data) AS Total_Reviews 
                    FROM Sentiment_Data 
                    GROUP BY Sentiment
                    ORDER BY Total_Reviews DESC
            8. Convert numerical outputs to float upto 1 decimal point.
            9. Always include ORDER BY clause to sort the table based on the aggregate value calculated in the query.
            10. Top Country is based on Sentiment_Score i.e., the Country which have highest sum(Sentiment_Score)
            11. Always use 'LIKE' operator whenever they mention about any Country. Use 'LIMIT' operator instead of TOP operator.Do not use TOP OPERATOR. Follow syntax that can be used with pandasql.
            12. If you are using any field in the aggregate function in select statement, make sure you add them in GROUP BY Clause.
            13. Make sure to Give the result as the query so that it can be used on Microsoft SQL server SSMS.
            14. Important: Always show Net_Sentiment in Percentage upto 1 decimal point. Hence always make use of ROUND function while giving out Net Sentiment and Add % Symbol after it.
            15. Important: User can ask question about any categories including Aspects, Geograpgy, Sentiment etc etc. Hence, include the in SQL Query if someone ask it.
            16. Important: You Response should directly starts from SQL query nothing else.
            17. Important: Always use LIKE keyword instead of = symbol while generating SQL query.
            18. Important: Generate outputs using the provided dataset only, don't use pre-trained information to generate outputs.
        \n Following is the previous conversation from User and Response, use it to get context only:""" + hist + """\n
                Use the above conversation chain to gain context if the current prompt requires context from previous conversation.\n
        Context:\n {context}?\n
        Question: \n{question}\n

        Answer:
        """
        prompt = PromptTemplate(template=prompt_template, input_variables=["context", "question"])
        model = AzureChatOpenAI(
            azure_deployment="Thruxton_R",
            api_version='2024-03-01-preview',
            temperature = 0.3)
        chain = load_qa_chain(model, chain_type="stuff", prompt=prompt)
        return chain

SyntaxError: invalid syntax (1269006424.py, line 1)

In [7]:
prompt_template = "How are you?"
prompt = PromptTemplate(template=prompt_template, input_variables=["context", "question"])
model = AzureChatOpenAI(
    azure_deployment="Thruxton_R",
    api_version='2024-03-01-preview',
    temperature = 0.3)
chain = load_qa_chain(model, chain_type="stuff", prompt=prompt)

ValidationError: 1 validation error for StuffDocumentsChain
__root__
  document_variable_name context was not found in llm_chain input_variables: [] (type=value_error)

In [1]:
#Import Required Libraries
import streamlit as st
from azure.core.credentials import AzureKeyCredential
from langchain.text_splitter import RecursiveCharacterTextSplitter
import numpy as np
import faiss
from langchain_community.vectorstores import FAISS
from langchain_core.vectorstores import VectorStoreRetriever
from langchain.chains import RetrievalQA
from langchain.llms import AzureOpenAI
from langchain_openai import AzureOpenAIEmbeddings
from langchain.prompts import PromptTemplate
from langchain.chains.question_answering import load_qa_chain
from langchain_core.messages import HumanMessage
from langchain_openai import AzureChatOpenAI
import openai
import pyodbc
import urllib
from sqlalchemy import create_engine
import pandas as pd
from azure.identity import InteractiveBrowserCredential
from pandasai import SmartDataframe
import pandas as pd
# from pandasai.llm import AzureOpenAI
import matplotlib.pyplot as plt
import os
import time
from PIL import Image
import base64
import pandasql as ps

In [2]:
os.environ["AZURE_OPENAI_API_KEY"] = "672370cd6ca440f2a0327351d4f4d2bf"
os.environ["AZURE_OPENAI_ENDPOINT"] = "https://hulk-openai.openai.azure.com/"

In [12]:
client = AzureOpenAI(
    api_key=os.getenv("672370cd6ca440f2a0327351d4f4d2bf"),  
    api_version="2024-02-01",
    azure_endpoint = os.getenv("https://hulk-openai.openai.azure.com/")
    )
    
deployment_name='SurfaceGenAI'


context = """
    1. Your Job is to convert the user question to SQL Query (Follow Microsoft SQL server SSMS syntax.). You have to give the query so that it can be used on Microsoft SQL server SSMS.You have to only return query as a result.
    2. There is only one table with table name RCR_Sales_Data where each row has. The table has 20 columns, they are:
        Month: Contains dates for the records
        Country: From where the sales has happened. It contains following values: 'Turkey','India','Brazil','Germany','Philippines','France','Netherlands','Spain','United Arab Emirates','Czech Republic','Norway','Belgium','Finland','Canada','Mexico','Russia','Austria','Poland','United States','Switzerland','Italy','Colombia','Japan','Chile','Sweden','Vietnam','Saudi Arabia','South Africa','Peru','Indonesia','Taiwan','Thailand','Ireland','Korea','Hong Kong SAR','Malaysia','Denmark','New Zealand','China' and 'Australia'.
        Geography: From which Country or Region the review was given. It contains following values: 'Unknown', 'Brazil', 'Australia', 'Canada', 'China', 'Germany','France'.
        OEMGROUP: OEM or Manufacturer of the Device. It contains following values: 'Lenovo','Acer','Asus','HP','All Other OEMs', 'Microsoft' and 'Samsung'
        SUBFORMFACTOR: Formfactor of the device. It contains following values: 'Ultraslim Notebook'.
        GAMINGPRODUCTS: Flag whether Device is a gaming device or not. It contains following values: 'GAMING', 'NO GAMING' and 'N.A.'.
        SCREEN_SIZE_INCHES: Screen Size of the Device.
        PRICE_BRAND_USD_3: Band of the price at which the device is selling. It contains following values: '0-300', '300-500', '500-800' and '800+.
        OS_VERSION: Operating System version intall on the device. It contains following values: 'Windows 11', 'Chrome', 'Mac OS'.
        Operating_System_Summary: Operating System installed on the device. This is at uber level. It contains following values: 'Windows', 'Google OS', 'Apple OS'.
        Sales_Units: Number of Devices sold for that device in a prticular month and country.
        Sales_Value: Revenue Generated by the devices sold.
        Series: Family of the device such as IdeaPad 1, HP Laptop 15 etc.
        Specs_Combination: Its contains the combination of Series, Processor, RAM , Storage and Screen Size. For Example: SURFACE LAPTOP GO | Ci5 | 8 GB | 256.0 SSD | 12" .
    3.  When Asked for Price Range you have to use ASP Column to get minimum and Maxium value. Do not consider Negative Values. Also Consider Sales Units it shouldn't be 0.
        Exaple Query:
            SELECT MIN(ASP) AS Lowest_Value, MAX(ASP) AS Highest_Value
            FROM RCR_Sales_Data
            WHERE
            Series = 'Device Name'
            AND ASP >= 0
            AND Sales_Units <> 0;
    4. Total Sales_Units Should Always be in Thousands. 
        Example Query:
            SELECT (SUM(Sales_Units) / 1000) AS "TOTAL SALES UNITS"
            FROM RCR_Sales_Data
            WHERE
            SERIES LIKE '%SURFACE LAPTOP GO%';
    5. Average Selling Price (ASP): It is calculated by sum of SUM(Sales_Value)/SUM(Sales_Units)
    6. Total Sales Units across countries or across regions is sum of sales_units for those country. It should be in thousand of million hence add "K" or "M" after the number.
        Example to calculate sales units across country:
            SELECT Country, (SUM(Sales_Units) / 1000) AS "Sales_Units(In Thousands)"
            FROM RCR_Sales_Data
            GROUP BY Country
            ORDER BY Sales_Units DESC
    7. Total Sales Units across column "X" or across regions is sum of sales_units for those country. It should be in thousand of million hence add "K" or "M" after the number.
        Example to calculate sales units across country:
            SELECT "X", (SUM(Sales_Units) / 1000) AS "Sales_Units(In Thousands)"
            FROM RCR_Sales_Data
            GROUP BY "X"
            ORDER BY Sales_Units DESC
    8. If asked about the highest selling Specs Combination. 
        Example Query:
            SELECT Specs_Combination, (SUM(Sales_Units) / 1000) AS "TOTAL SALES UNITS"
            FROM RCR_Sales_Data
            WHERE SERIES LIKE '%Macbook AIR%'
            AND SALES_UNITS <> 0
            GROUP BY Specs_Combination
            ORDER BY "TOTAL SALES UNITS" DESC
            LIMIT 1;
    9. If asked about similar compete devices.
    Example Query:
            WITH SurfaceLaptopGoASP AS (
                SELECT
                    'Surface Laptop Go' AS Series,
                    SUM(Sales_Value) / SUM(Sales_Units) AS ASP
                FROM
                    RCR_Sales_Data
                WHERE
                    Series LIKE '%Surface Laptop Go%'
            ),
            CompetitorASP AS (
                SELECT
                    Series,
                    SUM(Sales_Value) / SUM(Sales_Units) AS ASP
                FROM
                    RCR_Sales_Data
                WHERE
                    Operating_System_Summary IN ('Apple OS', 'Google OS')
                GROUP BY
                    Series
            )
            SELECT
                C.Series,
                C.ASP AS CompetitorASP
            FROM
                CompetitorASP C
            JOIN
                SurfaceLaptopGoASP S
            ON
                ABS(C.ASP - S.ASP) <= 200;
    10. If asked about dates or year SUBSTR() function instead of Year() or Month()
    11. Convert numerical outputs to float upto 2 decimal point.
    12. Always include ORDER BY clause to sort the table based on the aggregate value calculated in the query.
    13. Always use 'LIKE' operator whenever they mention about any Country, Series. Use 'LIMIT' operator instead of TOP operator.Do not use TOP OPERATOR. Follow syntax that can be used with pandasql.
    14. If you are using any field in the aggregate function in select statement, make sure you add them in GROUP BY Clause.
    15. Make sure to Give the result as the query so that it can be used on Microsoft SQL server SSMS.
    16. Always use LIKE function instead of = Symbol while generating SQL Query
    17. Important: User can ask question about any categories including Country, OEMGROUP,OS_VERSION etc etc. Hence, include the in SQL Query if someone ask it.
    18. Important: Use the correct column names listed above. There should not be Case Sensitivity issue. 
    19. Important: The values in OPERATING_SYSTEM_SUMMARY are ('Apple OS', 'Google OS') not ('APPLE OS', 'GOOGLE OS'). So use exact values. Not everything should be capital letters.
    20. Important: You Response should directly starts from SQL query nothing else."""
# Initialize an empty context

def generate_SQL_Query(user_question):
    global context
    # Append the new question to the context
    full_prompt = context + "\nQuestion:\n" + user_question + "\nAnswer:"
    
    # Send the query to Azure OpenAI
    response = client.completions.create(
        model=deployment_name,
        prompt=full_prompt,
        max_tokens=500,
        temperature=0
    )
    
    # Extract the generated SQL query
    sql_query = response.choices[0].text.strip()
    
    # Update context with the latest interaction
    context += "\nQuestion:\n" + user_question + "\nAnswer:\n" + sql_query
    
    return sql_query

In [13]:
#Converting Top Operator to Limit Operator as pandasql doesn't support Top
def convert_top_to_limit(sql):
    tokens = sql.upper().split()
    is_top_used = False

    for i, token in enumerate(tokens):
        if token == 'TOP':
            is_top_used = True
            if i + 1 < len(tokens) and tokens[i + 1].isdigit():
                limit_value = tokens[i + 1]
                # Remove TOP and insert LIMIT and value at the end
                del tokens[i:i + 2]
                tokens.insert(len(tokens), 'LIMIT')
                tokens.insert(len(tokens), limit_value)
                break  # Exit loop after successful conversion
            else:
                raise ValueError("TOP operator should be followed by a number")

    return ' '.join(tokens) if is_top_used else sql


def process_tablename(sql, table_name):
    x = sql.upper()
    query = x.replace(table_name.upper(), table_name)
    return query

In [14]:
RCR_Sales_Data = pd.read_csv('RCR Sales Data Sample V2.csv')
RCR_Sales_Data.columns

Index(['Month', 'Country', 'OEMGROUP', 'SUBFORMFACTOR', 'GAMINGPRODUCTS',
       'SCREEN_SIZE_INCHES', 'PRICE_BRAND_USD_3', 'OS_VERSION',
       'Operating_System_Summary', 'Sales_Units', 'Sales_Value', 'ASP',
       'Series', 'Specs_Combination'],
      dtype='object')

In [15]:
question = "What ASP for Surface Laptop Go"

In [210]:
a = generate_SQL_Query(question)

In [211]:
SQL_Query = convert_top_to_limit(a)
SQL_Query = process_tablename(SQL_Query,"RCR_Sales_Data")
print(SQL_Query)

SELECT (SUM(SALES_VALUE) / SUM(SALES_UNITS)) AS ASP
FROM RCR_Sales_Data
WHERE
SERIES LIKE '%SURFACE LAPTOP GO%'
AND ASP >= 0
AND SALES_UNITS <> 0;


In [212]:
data = ps.sqldf(SQL_Query, globals())

In [213]:
data

,ASP
0,653.236509


In [331]:
user_input = "How is Surface Laptop Go 2 performing in the market"

In [325]:
df = pd.read_csv('Device Images.csv')

In [335]:
def get_device_image():
    df = pd.read_csv('Device Images.csv')
    for i in df['Device Name']:
        if str.lower(i) in str.lower(user_input):
            dev = i
    link = df[df['Device Name']==dev]['Link'][0]
    print(link)

In [337]:
get_device_image()

https://m.media-amazon.com/images/I/41JM2AxnD9L.jpg


# Code to get sales units for a device

In [217]:
def get_sales_units(device_name):
    question = "Totals Sales Units for " + device_name
    a = generate_SQL_Query(question)
    SQL_Query = convert_top_to_limit(a)
    SQL_Query = process_tablename(SQL_Query,"RCR_Sales_Data")
    data = ps.sqldf(SQL_Query, globals())
    col_name = data.columns[0]
    total_sales = data[col_name][0]
    total_sales = str(round(total_sales,2)) + "K"
    return total_sales

In [218]:
abc = get_sales_units("Surface Laptop Go")
abc

'271.32K'

In [221]:
def get_ASP(device_name):
    question = "What's ASP for " + device_name
    a = generate_SQL_Query(question)
    SQL_Query = convert_top_to_limit(a)
    SQL_Query = process_tablename(SQL_Query,"RCR_Sales_Data")
    data = ps.sqldf(SQL_Query, globals())
    col_name = data.columns[0]
    asp = data[col_name][0]
    asp = "$" + str(int(round(asp,0)))
    return asp

In [222]:
abc = get_ASP("Surface Laptop Go")
abc

'$653'

In [16]:
def get_highest_selling_specs(device_name):
    question = "What's highest selling Specs Combination for " + device_name
    a = generate_SQL_Query(question)
    SQL_Query = convert_top_to_limit(a)
    SQL_Query = process_tablename(SQL_Query,"RCR_Sales_Data")
    data = ps.sqldf(SQL_Query, globals())
    col_name1 = data.columns[0]
    col_name2 = data.columns[1]
    specs = data[col_name1][0]
    sales_unit = data[col_name2][0]
    sales_unit = str(round(sales_unit,2)) + "K"
    return specs,sales_unit

In [17]:
abc,su = get_highest_selling_specs("Macbook Air")
print(abc," - ",su)

MACBOOK AIR M1 | M1 | 8 GB | 256 SSD | 13"  -  2621.18K


In [322]:
def compete_device(device_name):
    question = "What are the compete device for " + device_name
    a = generate_SQL_Query(question)
    SQL_Query = convert_top_to_limit(a)
    SQL_Query = process_tablename(SQL_Query,"RCR_Sales_Data")
    SQL_Query = SQL_Query.replace('APPLE','Apple')
    SQL_Query = SQL_Query.replace('GOOGLE','Google')
    data = ps.sqldf(SQL_Query, globals())
    col_name1 = data.columns[0]
    devices = list(data[col_name1])
    return devices

In [31]:
abc = compete_device("Surface Laptop Go")
abc

NameError: name 'compete_device' is not defined

In [6]:
client = AzureOpenAI(
    api_key=os.getenv("672370cd6ca440f2a0327351d4f4d2bf"),  
    api_version="2024-02-01",
    azure_endpoint = os.getenv("https://hulk-openai.openai.azure.com/")
    )
    
deployment_name='SurfaceGenAI'

In [28]:
detail_summary_template_prompt = """Provide a detailed consumer review summary for the [device_name] with aspect-wise net sentiment. Please mention what consumers like and dislike about the device, focusing on Performance, Design, and Display.
Overall, the [device_name] is well-received by consumers, with its performance, design, and display being the standout features. However, there are some minor criticisms regarding performance issues and display brightness.
Aspect: Performance
Net Sentiment: +70%
Likes: Users appreciate the smooth performance of the device, noting its fast processing speed and ability to handle multitasking with ease.
Dislikes: Some users have reported occasional lag or slowdowns, especially when running demanding applications or games.

Aspect: Design
Net Sentiment: +85%
Likes: Consumers love the sleek and compact design of the device, praising its lightweight build and premium look and feel.
Dislikes: A few users find the design too simplistic and wish for more color options or customizable features.

Aspect: Display
Net Sentiment: +75%
Likes: Users are impressed with the vibrant and sharp display of the device, noting its accurate colors and wide viewing angles.
Dislikes: Some users feel that the display could be brighter, especially when using the device outdoors or in brightly lit environments.

Mention the need for Improvement as well in detail"""

In [23]:
response = client.completions.create(
        model=deployment_name,
        prompt=template_prompt,
        max_tokens=1000,
        temperature=0
    )

In [5]:
from openai import AzureOpenAI

In [24]:
response.choices[0].text

"\n\n\nThe Microsoft Surface Laptop Go has received mixed reviews from consumers, with a net sentiment of 60%. While some aspects of the device have been praised, others have been criticized.\n\nPerformance:\nThe performance of the Surface Laptop Go has received a positive sentiment of 70%. Many users have appreciated the device's fast processing speed and smooth performance. The 10th generation Intel Core processor has been praised for its ability to handle multiple tasks without any lag. However, some users have reported occasional crashes and slow boot-up times, which has brought down the overall sentiment.\n\nDesign:\nThe design of the Surface Laptop Go has received a high sentiment of 80%. The sleek and lightweight design has been praised by users, making it easy to carry around. The device also has a premium look and feel, with its aluminum chassis and vibrant color options. However, some users have expressed disappointment with the lack of ports, as the device only has one USB-C

In [29]:
def get_detailed_summary(user_imput):
    response = client.completions.create(
        model=deployment_name,
        prompt=template_prompt+user_imput,
        max_tokens=1000,
        temperature=0.2
    )
    output = response.choices[0].text
    return output

In [30]:
a = get_detailed_summary("GIve me detailed Summary for Surface Laptop Go 2")
a

'\n\nOverall, the Surface Laptop Go 2 has received positive reviews from consumers, with its performance, design, and display being the standout features. However, there are some areas that could use improvement, such as battery life and storage capacity.\n\nAspect: Performance\nNet Sentiment: +80%\nLikes: Users are impressed with the performance of the Surface Laptop Go 2, noting its fast processing speed and ability to handle multiple tasks without any lag.\nDislikes: Some users have reported occasional slowdowns or crashes, especially when running demanding applications or multitasking with multiple programs open.\n\nAspect: Design\nNet Sentiment: +90%\nLikes: Consumers love the sleek and lightweight design of the Surface Laptop Go 2, praising its portability and premium look and feel.\nDislikes: A few users wish for more color options or a more durable build, as some have reported minor scratches or dents on the device.\n\nAspect: Display\nNet Sentiment: +75%\nLikes: Users apprecia

In [46]:
def get_device_image(user_input):
    print("User I",user_input)
    df = pd.read_csv('Device Images.csv')
    for i in df['Device Name']:
        print(i)
        if str.lower(i) in str.lower(user_input):
            dev = i
            link = df[df['Device Name']==dev]['Link'][0]
        else:
            dev = None
            link = None
    return (dev, link)

In [47]:
comp_devices = ['CHROMEBOOK FLEX 5',
                'HP CHROMEBOOK X2',
                'IDEAPAD CHROMEBOOK FLEX',
                'MACBOOK AIR',
                'MACBOOK AIR INTEL']

In [50]:
import pandas as pd

comp_devices = ['CHROMEBOOK FLEX 5',
                'HP CHROMEBOOK X2',
                'IDEAPAD CHROMEBOOK FLEX',
                'MACBOOK AIR',
                'MACBOOK AIR INTEL']

def get_device_image(user_input):
    df = pd.read_csv('Device Images.csv')
    dev = None
    for i in df['Device Name']:
        if str.lower(i) in str.lower(user_input):
            dev = i
            break  # Exit the loop once a match is found
    if dev is None:
        return None, None  # Return None if no matching device is found
    link = df[df['Device Name']==dev]['Link'].values[0]  # Using .values[0] to get the link
    return dev, link

device_links = {}
for device in comp_devices:
    dev, link = get_device_image(device)
    if dev is not None:
        device_links[dev] = link

print(device_links)

{'Macbook Air': 'https://store.storeimages.cdn-apple.com/4668/as-images.apple.com/is/mba13-midnight-select-202402?wid=904&hei=840&fmt=jpeg&qlt=90&.v=1708367688034'}


In [69]:
from QuantitativeSummaryFinal import Sentiment_Score_Derivation, get_final_df,custom_color_gradient, query_detailed, get_conversational_chain_detailed, query_detailed_summary, get_conversational_chain_detailed_summary, query_quant, get_conversational_chain_quant, process_tablename

In [77]:
device_name = "Hp laptop 15"
device = device_name

def get_detailed_summary(device_name):
    if device_name:
        data = query_quant("Summarize the reviews of "+ device_name, [])
        total_reviews = data.loc[data['ASPECT'] == 'TOTAL', 'REVIEW_COUNT'].iloc[0]
        data['REVIEW_PERCENTAGE'] = data['REVIEW_COUNT'] / total_reviews * 100
        dataframe_as_dict = data.to_dict(orient='records')
        data_new = data
        data_new = data_new.dropna(subset=['ASPECT_SENTIMENT'])
        data_new = data_new[~data_new["ASPECT"].isin(["Generic", "Account", "Customer-Service", "Browser"])]
        vmin = data_new['ASPECT_SENTIMENT'].min()
        vmax = data_new['ASPECT_SENTIMENT'].max()
        styled_df = data_new.style.applymap(lambda x: custom_color_gradient(x, vmin, vmax), subset=['ASPECT_SENTIMENT'])
        data_filtered = data_new[data_new['ASPECT'] != 'TOTAL']
        data_sorted = data_filtered.sort_values(by='REVIEW_COUNT', ascending=False)
        top_four_aspects = data_sorted.head(4)
        aspects_list = top_four_aspects['ASPECT'].to_list()
        aspects_list
        formatted_aspects = ', '.join(f"'{aspect}'" for aspect in aspects_list)
        key_df = get_final_df(aspects_list, device_name)
        b =  key_df.to_dict(orient='records')
        su = query_detailed_summary("Summarize reviews of" + device + "for " +  formatted_aspects +  "Aspects which have following "+str(dataframe_as_dict)+ str(b) + "Reviews: ",[])
    return su

In [78]:
a = get_detailed_summary("Surface Laptop Go")

In [66]:
Sentiment_Data = pd.read_csv('Windows_Data_116K.csv')

In [6]:
#Import Required Libraries
import streamlit as st
from azure.core.credentials import AzureKeyCredential
from langchain.text_splitter import RecursiveCharacterTextSplitter
import numpy as np
import faiss
from langchain_community.vectorstores import FAISS
from langchain_core.vectorstores import VectorStoreRetriever
from langchain.chains import RetrievalQA
# from langchain.llms import AzureOpenAI
from langchain_openai import AzureOpenAIEmbeddings
from langchain.prompts import PromptTemplate
from langchain.chains.question_answering import load_qa_chain
from langchain_core.messages import HumanMessage
from langchain_openai import AzureChatOpenAI
import openai
import pyodbc
import urllib
from sqlalchemy import create_engine
import pandas as pd
from azure.identity import InteractiveBrowserCredential
from pandasai import SmartDataframe
import pandas as pd
# from pandasai.llm import AzureOpenAI
import matplotlib.pyplot as plt
import os
import time
from PIL import Image
import base64
import pandasql as ps
from openai import AzureOpenAI
from QuantitativeSummaryFinal import Sentiment_Score_Derivation, get_final_df,custom_color_gradient, query_detailed, get_conversational_chain_detailed, query_detailed_summary, get_conversational_chain_detailed_summary, query_quant, get_conversational_chain_quant, process_tablename
os.environ['KMP_DUPLICATE_LIB_OK'] = 'TRUE'
os.environ["AZURE_OPENAI_API_KEY"] = "672370cd6ca440f2a0327351d4f4d2bf"
os.environ["AZURE_OPENAI_ENDPOINT"] = "https://hulk-openai.openai.azure.com/"


client = AzureOpenAI(
    api_key=os.getenv("672370cd6ca440f2a0327351d4f4d2bf"),  
    api_version="2024-02-01",
    azure_endpoint = os.getenv("https://hulk-openai.openai.azure.com/")
    )
    
deployment_name='SurfaceGenAI'


context = """
    1. Your Job is to convert the user question to SQL Query (Follow Microsoft SQL server SSMS syntax.). You have to give the query so that it can be used on Microsoft SQL server SSMS.You have to only return query as a result.
    2. There is only one table with table name RCR_Sales_Data where each row has. The table has 20 columns, they are:
        Month: Contains dates for the records
        Country: From where the sales has happened. It contains following values: 'Turkey','India','Brazil','Germany','Philippines','France','Netherlands','Spain','United Arab Emirates','Czech Republic','Norway','Belgium','Finland','Canada','Mexico','Russia','Austria','Poland','United States','Switzerland','Italy','Colombia','Japan','Chile','Sweden','Vietnam','Saudi Arabia','South Africa','Peru','Indonesia','Taiwan','Thailand','Ireland','Korea','Hong Kong SAR','Malaysia','Denmark','New Zealand','China' and 'Australia'.
        Geography: From which Country or Region the review was given. It contains following values: 'Unknown', 'Brazil', 'Australia', 'Canada', 'China', 'Germany','France'.
        OEMGROUP: OEM or Manufacturer of the Device. It contains following values: 'Lenovo','Acer','Asus','HP','All Other OEMs', 'Microsoft' and 'Samsung'
        SUBFORMFACTOR: Formfactor of the device. It contains following values: 'Ultraslim Notebook'.
        GAMINGPRODUCTS: Flag whether Device is a gaming device or not. It contains following values: 'GAMING', 'NO GAMING' and 'N.A.'.
        SCREEN_SIZE_INCHES: Screen Size of the Device.
        PRICE_BRAND_USD_3: Band of the price at which the device is selling. It contains following values: '0-300', '300-500', '500-800' and '800+.
        OS_VERSION: Operating System version intall on the device. It contains following values: 'Windows 11', 'Chrome', 'Mac OS'.
        Operating_System_Summary: Operating System installed on the device. This is at uber level. It contains following values: 'Windows', 'Google OS', 'Apple OS'.
        Sales_Units: Number of Devices sold for that device in a prticular month and country.
        Sales_Value: Revenue Generated by the devices sold.
        Series: Family of the device such as IdeaPad 1, HP Laptop 15 etc.
        Specs_Combination: Its contains the combination of Series, Processor, RAM , Storage and Screen Size. For Example: SURFACE LAPTOP GO | Ci5 | 8 GB | 256.0 SSD | 12" .
        Chassis_Segment: It contains following values: 'SMB_Upper','Mainstream_Lower','SMB_Lower','Enterprise Fleet_Lower','Entry','Mainstream_Upper','Premium Mobility_Upper','Enterprise Fleet_Upper','Premium Mobility_Lower','Creation_Lower','UNDEFINED','Premium_Mobility_Upper','Enterprise Work Station','Unknown','Gaming_Musclebook','Entry_Gaming','Creation_Upper','Mainstrean_Lower'
    3.  When Asked for Price Range you have to use ASP Column to get minimum and Maxium value. Do not consider Negative Values. Also Consider Sales Units it shouldn't be 0.
        Exaple Query:
            SELECT MIN(ASP) AS Lowest_Value, MAX(ASP) AS Highest_Value
            FROM RCR_Sales_Data
            WHERE
            Series = 'Device Name'
            AND ASP >= 0
            AND Sales_Units <> 0;
    4. Total Sales_Units Should Always be in Thousands. 
        Example Query:
            SELECT (SUM(Sales_Units) / 1000) AS "TOTAL SALES UNITS"
            FROM RCR_Sales_Data
            WHERE
            SERIES LIKE '%SURFACE LAPTOP GO%';
    5. Average Selling Price (ASP): It is calculated by sum of SUM(Sales_Value)/SUM(Sales_Units)
    6. Total Sales Units across countries or across regions is sum of sales_units for those country. It should be in thousand of million hence add "K" or "M" after the number.
        Example to calculate sales units across country:
            SELECT Country, (SUM(Sales_Units) / 1000) AS "Sales_Units(In Thousands)"
            FROM RCR_Sales_Data
            GROUP BY Country
            ORDER BY Sales_Units DESC
    7. Total Sales Units across column "X" or across regions is sum of sales_units for those country. It should be in thousand of million hence add "K" or "M" after the number.
        Example to calculate sales units across country:
            SELECT "X", (SUM(Sales_Units) / 1000) AS "Sales_Units(In Thousands)"
            FROM RCR_Sales_Data
            GROUP BY "X"
            ORDER BY Sales_Units DESC
    8. If asked about the highest selling Specs Combination. 
        Example Query:
            SELECT Specs_Combination, (SUM(Sales_Units) / 1000) AS "TOTAL SALES UNITS"
            FROM RCR_Sales_Data
            WHERE SERIES LIKE '%Macbook AIR%'
            AND SALES_UNITS <> 0
            GROUP BY Specs_Combination
            ORDER BY "TOTAL SALES UNITS" DESC
            LIMIT 1;
    9. If asked about similar compete devices.
    Example Query: WITH DeviceNameASP AS (
                SELECT
                    'Device Name' AS Series,
                    SUM(Sales_Value) / SUM(Sales_Units) AS ASP
                FROM
                    RCR_Sales_Data
                WHERE
                    Series LIKE '%Device Name%'
            ),
            CompetitorASP AS (
                SELECT
                    Series,
                    SUM(Sales_Value) / SUM(Sales_Units) AS ASP
                FROM
                    RCR_Sales_Data
                WHERE
                    Operating_System_Summary IN ('Apple OS', 'Google OS')
                GROUP BY
                    Series
            )
            SELECT
                C.Series,
                C.ASP AS CompetitorASP
            FROM
                CompetitorASP C
            JOIN
                SurfaceLaptopGoASP S
            ON
                ABS(C.ASP - S.ASP) <= 200;
    10. If asked about dates or year SUBSTR() function instead of Year() or Month()
    11. Convert numerical outputs to float upto 2 decimal point.
    12. Always include ORDER BY clause to sort the table based on the aggregate value calculated in the query.
    13. Always use 'LIKE' operator whenever they mention about any Country, Series. Use 'LIMIT' operator instead of TOP operator.Do not use TOP OPERATOR. Follow syntax that can be used with pandasql.
    14. If you are using any field in the aggregate function in select statement, make sure you add them in GROUP BY Clause.
    15. Make sure to Give the result as the query so that it can be used on Microsoft SQL server SSMS.
    16. Always use LIKE function instead of = Symbol while generating SQL Query
    17. Important: User can ask question about any categories including Country, OEMGROUP,OS_VERSION etc etc. Hence, include the in SQL Query if someone ask it.
    18. Important: Use the correct column names listed above. There should not be Case Sensitivity issue. 
    19. Important: The values in OPERATING_SYSTEM_SUMMARY are ('Apple OS', 'Google OS') not ('APPLE OS', 'GOOGLE OS'). So use exact values. Not everything should be capital letters.
    20. Important: You Response should directly starts from SQL query nothing else."""
# Initialize an empty context
detail_summary_template_prompt = """Provide a detailed consumer review summary for the [device_name] with aspect-wise net sentiment. Please mention what consumers like and dislike about the device, focusing on Performance, Design, and Display.
Overall, the [device_name] is well-received by consumers, with its performance, design, and display being the standout features. However, there are some minor criticisms regarding performance issues and display brightness.
Aspect: Performance
Net Sentiment: +70%
Likes: Users appreciate the smooth performance of the device, noting its fast processing speed and ability to handle multitasking with ease.
Dislikes: Some users have reported occasional lag or slowdowns, especially when running demanding applications or games.

Aspect: Design
Net Sentiment: +85%
Likes: Consumers love the sleek and compact design of the device, praising its lightweight build and premium look and feel.
Dislikes: A few users find the design too simplistic and wish for more color options or customizable features.

Aspect: Display
Net Sentiment: +75%
Likes: Users are impressed with the vibrant and sharp display of the device, noting its accurate colors and wide viewing angles.
Dislikes: Some users feel that the display could be brighter, especially when using the device outdoors or in brightly lit environments.

Mention the need for Improvement as well in detail"""
def generate_SQL_Query(user_question):
    global context
    # Append the new question to the context
    full_prompt = context + "\nQuestion:\n" + user_question + "\nAnswer:"
    
    # Send the query to Azure OpenAI
    response = client.completions.create(
        model=deployment_name,
        prompt=full_prompt,
        max_tokens=500,
        temperature=0
    )
    
    # Extract the generated SQL query
    sql_query = response.choices[0].text.strip()
    
    # Update context with the latest interaction
    context += "\nQuestion:\n" + user_question + "\nAnswer:\n" + sql_query
    
    return sql_query

#Converting Top Operator to Limit Operator as pandasql doesn't support Top
def convert_top_to_limit(sql):
    tokens = sql.upper().split()
    is_top_used = False

    for i, token in enumerate(tokens):
        if token == 'TOP':
            is_top_used = True
            if i + 1 < len(tokens) and tokens[i + 1].isdigit():
                limit_value = tokens[i + 1]
                # Remove TOP and insert LIMIT and value at the end
                del tokens[i:i + 2]
                tokens.insert(len(tokens), 'LIMIT')
                tokens.insert(len(tokens), limit_value)
                break  # Exit loop after successful conversion
            else:
                raise ValueError("TOP operator should be followed by a number")

    return ' '.join(tokens) if is_top_used else sql


def process_tablename(sql, table_name):
    x = sql.upper()
    query = x.replace(table_name.upper(), table_name)
    return query

RCR_Sales_Data = pd.read_csv('RCR Sales Data Sample V2.csv')


def get_sales_units(device_name):
    question = "Totals Sales Units for " + device_name
    a = generate_SQL_Query(question)
    SQL_Query = convert_top_to_limit(a)
    SQL_Query = process_tablename(SQL_Query,"RCR_Sales_Data")
    data = ps.sqldf(SQL_Query, globals())
    col_name = data.columns[0]
    total_sales = data[col_name][0]
    total_sales = str(round(total_sales,2)) + "K"
    return total_sales


def get_ASP(device_name):
    question = "What's ASP for " + device_name
    a = generate_SQL_Query(question)
    SQL_Query = convert_top_to_limit(a)
    SQL_Query = process_tablename(SQL_Query,"RCR_Sales_Data")
    data = ps.sqldf(SQL_Query, globals())
    col_name = data.columns[0]
    asp = data[col_name][0]
    asp = "$" + str(int(round(asp,0)))
    return asp

def get_highest_selling_specs(device_name):
    question = "What's highest selling Specs Combination for " + device_name
    a = generate_SQL_Query(question)
    SQL_Query = convert_top_to_limit(a)
    SQL_Query = process_tablename(SQL_Query,"RCR_Sales_Data")
    data = ps.sqldf(SQL_Query, globals())
    col_name1 = data.columns[0]
    col_name2 = data.columns[1]
    specs = data[col_name1][0]
    sales_unit = data[col_name2][0]
    sales_unit = str(round(sales_unit,2)) + "K"
    return specs,sales_unit

def compete_device(device_name):
    question = "What are the compete device for " + device_name
    a = generate_SQL_Query(question)
    SQL_Query = convert_top_to_limit(a)
    SQL_Query = process_tablename(SQL_Query,"RCR_Sales_Data")
    SQL_Query = SQL_Query.replace('APPLE','Apple')
    SQL_Query = SQL_Query.replace('GOOGLE','Google')
    print(SQL_Query)
    data = ps.sqldf(SQL_Query, globals())
    col_name1 = data.columns[0]
    devices = list(data[col_name1])
    return devices
    
def get_detailed_summary(user_imput):
    response = client.completions.create(
        model=deployment_name,
        prompt=detail_summary_template_prompt+user_imput,
        max_tokens=1000,
        temperature=0.2
    )
    output = response.choices[0].text
    return output

def get_device_image(user_input):
    df = pd.read_csv('Device Images.csv')
    for i in df['Device Name']:
        if str.lower(i) in str.lower(user_input):
            dev = i
    link = df[df['Device Name']==dev]['Link'].values[0]
    return (dev, link)

def get_comp_device_image(user_input):
    df = pd.read_csv('Device Images.csv')
    dev = None
    for i in df['Device Name']:
        if str.lower(i) in str.lower(user_input):
            dev = i
            break  # Exit the loop once a match is found
    if dev is None:
        return None, None  # Return None if no matching device is found
    link = df[df['Device Name']==dev]['Link'].values[0]  # Using .values[0] to get the link
    return dev, link
    
def get_detailed_summary(device_name):
    if device_name:
        data = query_quant("Summarize the reviews of "+ device_name, [])
        total_reviews = data.loc[data['ASPECT'] == 'TOTAL', 'REVIEW_COUNT'].iloc[0]
        data['REVIEW_PERCENTAGE'] = data['REVIEW_COUNT'] / total_reviews * 100
        dataframe_as_dict = data.to_dict(orient='records')
        data_new = data
        data_new = data_new.dropna(subset=['ASPECT_SENTIMENT'])
        data_new = data_new[~data_new["ASPECT"].isin(["Generic", "Account", "Customer-Service", "Browser"])]
        vmin = data_new['ASPECT_SENTIMENT'].min()
        vmax = data_new['ASPECT_SENTIMENT'].max()
        styled_df = data_new.style.applymap(lambda x: custom_color_gradient(x, vmin, vmax), subset=['ASPECT_SENTIMENT'])
        data_filtered = data_new[data_new['ASPECT'] != 'TOTAL']
        data_sorted = data_filtered.sort_values(by='REVIEW_COUNT', ascending=False)
        top_four_aspects = data_sorted.head(4)
        aspects_list = top_four_aspects['ASPECT'].to_list()
        formatted_aspects = ', '.join(f"'{aspect}'" for aspect in aspects_list)
        key_df = get_final_df(aspects_list, device_name)
        b =  key_df.to_dict(orient='records')
        su = query_detailed_summary("Summarize reviews of" + device_name + "for " +  formatted_aspects +  "Aspects which have following "+str(dataframe_as_dict)+ str(b) + "Reviews: ",[])
    return su
    
# def main():
#     try:
#     # Chat history state management
#         if 'chat_history' not in st.session_state:
#             st.session_state['chat_history'] = []

#         # Create a container for logos and title with horizontal layout
#         col1, col2, col3 = st.columns([1, 2, 1])
      
#         # Display logo on the left
#         with col1:
#             st.image("microsoft_logo.png", width=50)  # Adjust width as needed

#         # Display title in the center
#         with col2:
#             st.header("Consumer Reviews Synthesizer")

#         # Display logo on the right
#         with col3:
#             st.image("copilot_logo.svg", width=50)  # Align the logo to the right
      
#         # User input section
#         user_input = st.text_input("Enter your text:", placeholder="What would you like to process?")
#         if st.button("Process"):
#             sales_info = """
#             Total Devices Sold: 695.39K<br>
#             Average Selling Price: $1190<br>
#             Highest Selling Specs: SURFACE PRO | Ci5 | 8 GB | 256 SSD | 13" - 171.18K
#             """
#             device_name, img_link = get_device_image(user_input)
#             total_sales = get_sales_units(device_name)
#             asp = get_ASP(device_name)
#             high_specs, sale = get_highest_selling_specs(device_name)
#             html_code = f"""
#             <div style="background-color: white; padding: 20px; border-radius: 10px; box-shadow: 0px 0px 10px rgba(0, 0, 0, 0.1); display: flex; align-items: center;">
#                 <div style="flex: 1; text-align: center;">
#                     <img src="{img_link}" style="width: 150px; display: block; margin: 0 auto;">
#                     <p style="color: black; font-size: 18px;">{device_name}</p>
#                 </div>
#                 <div style="width: 2px; height: 150px; border-left: 2px dotted #ccc; margin: 0 20px;"></div>
#                 <div style="flex: 2; color: black; font-size: 18px;">
#                     <p>Total Devices Sold: <strong>{total_sales}</strong></p>
#                     <p>Average Selling Price: <strong>{asp}</strong></p>
#                     <p>Highest Selling Specs: <strong>{high_specs}</strong> - <strong>{sale}</strong></p>
#                 </div>
#             </div>
#             """
#             st.markdown(html_code, unsafe_allow_html=True)
            
#             st.write(r"$\textsf{\Large Detailed Summary}$")
#             summ = get_detailed_summary(user_input)
#             st.write(summ)
#             # st.write(r"$\textsf{\Large Compete Devices}$")
#             comp_devices = compete_device(device_name)
#             device_links = {}
#             for device in comp_devices:
#                 dev, link = get_comp_device_image(device)
#                 if dev is not None:
#                     device_links[dev] = link
            
#             col7, col8, col9 = st.columns([1, 2, 1])
#             for com_device_name, link in device_links.items():
#                 with col7:
#                     st.image(link,width = 150)
#                     if st.button(com_device_name):
#                         col10,col11 = st.columns([2,2])
#                         with col10:
#                             st.image(img_link,width = 150)
#                             st.write(device_name)
#                         with col11:
#                             st.image(link,width = 150)
#                             st.write(com_device_name)
                            
                
                                     
#     except Exception as e:
#         err = f"An error occurred while calling the final function: {e}"
#         print(err)
#         return err

# if __name__ == "__main__":
#     main()

In [7]:
compete_device("Microsoft Surface Pro")

WITH SURFACEPROASP AS (
                SELECT
                    'SURFACE PRO' AS SERIES,
                    SUM(SALES_VALUE) / SUM(SALES_UNITS) AS ASP
                FROM
                    RCR_Sales_Data
                WHERE
                    SERIES LIKE '%SURFACE PRO%'
            ),
            COMPETITORASP AS (
                SELECT
                    SERIES,
                    SUM(SALES_VALUE) / SUM(SALES_UNITS) AS ASP
                FROM
                    RCR_Sales_Data
                WHERE
                    OPERATING_SYSTEM_SUMMARY IN ('Apple OS', 'Google OS')
                GROUP BY
                    SERIES
            )
            SELECT
                C.SERIES,
                C.ASP AS COMPETITORASP
            FROM
                COMPETITORASP C
            JOIN
                SURFACEPROASP S
            ON
                ABS(C.ASP - S.ASP) <= 200;


['MACBOOK AIR M2', 'MACBOOK PRO 13']

In [36]:
SQL = """WITH DeviceNameASP AS (
    SELECT
        'Surface Pro' AS Series,
        SUM(Sales_Value) / SUM(Sales_Units) AS ASP,
        Chassis_Segment,
        SUM(Sales_Units) AS Sales_Units
    FROM
        RCR_Sales_Data
    WHERE
        Series LIKE '%Surface Pro%'
    GROUP BY
        Chassis_Segment
),
CompetitorASP AS (
    SELECT
        Series,
        SUM(Sales_Value) / SUM(Sales_Units) AS ASP,
        Chassis_Segment,
        SUM(Sales_Units) AS Sales_Units
    FROM
        RCR_Sales_Data
    WHERE
        Operating_System_Summary IN ('Apple OS', 'Google OS','Windows OS')
        AND SERIES NOT LIKE '%SURFACE PRO%'
    GROUP BY
        Series, Chassis_Segment
),
RankedCompetitors AS (
    SELECT
        C.Series,
        C.ASP,
        C.Chassis_Segment,
        C.Sales_Units,
        ROW_NUMBER() OVER (PARTITION BY C.Chassis_Segment ORDER BY C.Sales_Units DESC) AS rank
    FROM
        CompetitorASP C
    JOIN
        DeviceNameASP S
    ON
        ABS(C.ASP - S.ASP) <= 100
        AND C.Chassis_Segment = S.Chassis_Segment
)
SELECT
    Series,
    ASP AS CompetitorASP,
    Sales_Units
FROM
    RankedCompetitors
WHERE
    rank <= 4;"""


In [32]:
SQL_Query = convert_top_to_limit(SQL)
SQL_Query = process_tablename(SQL_Query,"RCR_Sales_Data")
SQL_Query = SQL_Query.replace('APPLE','Apple')
SQL_Query = SQL_Query.replace('GOOGLE','Google')
SQL_Query = SQL_Query.replace('WINDOWS','Windows')
print(SQL_Query)
data = ps.sqldf(SQL_Query, globals())

WITH DEVICENAMEASP AS (
    SELECT
        'SURFACE PRO' AS SERIES,
        SUM(SALES_VALUE) / SUM(SALES_UNITS) AS ASP,
        CHASSIS_SEGMENT,
        SUM(SALES_UNITS) AS SALES_UNITS
    FROM
        RCR_Sales_Data
    WHERE
        SERIES LIKE '%SURFACE PRO%'
    GROUP BY
        CHASSIS_SEGMENT
),
COMPETITORASP AS (
    SELECT
        SERIES,
        SUM(SALES_VALUE) / SUM(SALES_UNITS) AS ASP,
        CHASSIS_SEGMENT,
        SUM(SALES_UNITS) AS SALES_UNITS
    FROM
        RCR_Sales_Data
    WHERE
        OPERATING_SYSTEM_SUMMARY IN ('Apple OS', 'Google OS','Windows OS')
        AND SERIES NOT LIKE '%SURFACE PRO%'
    GROUP BY
        SERIES, CHASSIS_SEGMENT
),
RANKEDCOMPETITORS AS (
    SELECT
        C.SERIES,
        C.ASP,
        C.CHASSIS_SEGMENT,
        C.SALES_UNITS,
        ROW_NUMBER() OVER (PARTITION BY C.CHASSIS_SEGMENT ORDER BY C.SALES_UNITS DESC) AS RANK
    FROM
        COMPETITORASP C
    JOIN
        DEVICENAMEASP S
    ON
        ABS(C.ASP - S.ASP) <= 100
      

In [12]:
RCR_Sales_Data = pd.read_csv('RCR Sales Data Sample V3.csv')

In [33]:
data

,SERIES,COMPETITORASP,SALES_UNITS
0,MACBOOK AIR M2,1265.175216,4633171.84
1,SURFACE LAPTOP,1090.244301,412640.10
2,ENVY 17,1123.894939,93183.19
3,SPECTRE X360,1156.078007,66308.91


In [34]:
data

,SERIES,COMPETITORASP,SALES_UNITS
0,MACBOOK AIR M2,1265.175216,4633171.84
1,SURFACE LAPTOP,1090.244301,412640.10
2,ENVY 17,1123.894939,93183.19
3,SPECTRE X360,1156.078007,66308.91


In [37]:
def compete_device(device_name):
    # question = "What are the compete device for " + device_name
    # a = generate_SQL_Query(question)
    SQL_Query = convert_top_to_limit(SQL)
    SQL_Query = process_tablename(SQL_Query,"RCR_Sales_Data")
    SQL_Query = SQL_Query.replace('APPLE','Apple')
    SQL_Query = SQL_Query.replace('GOOGLE','Google')
    SQL_Query = SQL_Query.replace('WINDOWS','Windows')
    data = ps.sqldf(SQL_Query, globals())
    return data

In [38]:
df1 = compete_device("Microsoft Surface Pro")

In [39]:
df1

,SERIES,COMPETITORASP,SALES_UNITS
0,MACBOOK AIR M2,1265.175216,4633171.84
1,SURFACE LAPTOP,1090.244301,412640.10
2,ENVY 17,1123.894939,93183.19
3,SPECTRE X360,1156.078007,66308.91


In [62]:
def get_comp_device_details(user_input, df1):
    df = pd.read_csv('Device Images.csv')
    dev = None
    for i in df['Device Name']:
        print(i)
        if str.lower(i) in str.lower(user_input):
            dev = i
            break  # Exit the loop once a match is found
    
    if dev is None:
        return None, None, None, None  # Return None if no matching device is found
    
    link = df[df['Device Name'] == dev]['Link'].values[0]  # Using .values[0] to get the link
    df1['SERIES'] = df1['SERIES'].str.upper()
    dev = dev.upper()
    sales_data = df1[df1['SERIES'] == dev]
    if sales_data.empty:
        return dev, link, None, None  # Return dev and link, but None for sales and ASP if no matching SERIES is found
    
    sales = str(round(float(sales_data['SALES_UNITS'].values[0]) / 1000, 2)) + "K"
    ASP = "$" + str(int(sales_data['COMPETITORASP'].values[0]))
    
    return dev, link, sales, ASP

In [67]:
def get_net_sentiment(device_name):
    a = query_quant(device_name,[])
    try:
        Net_Sentiment = float(a[a['ASPECT']=='TOTAL']['ASPECT_SENTIMENT'].values[0])
    except:
        Net_Sentiment = None
    return Net_Sentiment

In [68]:
a = get_net_sentiment('Macbook Air M2')
a

  ASPECT ASPECT_SENTIMENT REVIEW_COUNT
0  TOTAL             None         None


TypeError: float() argument must be a string or a real number, not 'NoneType'

In [64]:
get_comp_device_details("macbook Air M2", df1)

Microsoft Surface Pro
Microsoft Surface Book 3 
Acer Aspire
Microsoft Surface Go 3 
Mackbook Pro 13
Microsoft Surface Laptop Go 
Microsoft Surface Pro 7 
hp pavilion 15
dell inspiron 15
lenovo ideapad slim 3
Microsoft Surface Go 2
Microsoft Surface Laptop 3
Microsoft Surface Laptop 4
Microsoft Surface Laptop 5
Microsoft Surface Laptop Studio
Microsoft Surface Pro 7 
Microsoft Surface Pro 8
Microsoft Surface Pro 9
Microsoft Surface Studio 2
Macbook Air M2


('MACBOOK AIR M2',
 'https://store.storeimages.cdn-apple.com/4668/as-images.apple.com/is/mba13-midnight-select-202402?wid=904&hei=840&fmt=jpeg&qlt=90&.v=1708367688034',
 '4633.17K',
 '$1265')

In [ ]:
comp_devices = compete_device(device_name)
device_links = {}
for device in comp_devices:
    dev, link = get_comp_device_details(device)
    if dev is not None:
        device_links[dev] = link